# Fundamental 17. Tensorflow2 API

## Tensorflow2 API로 모델 구성: MNIST 데이터셋

###  MNIST 데이터셋 : Sequential API 활용

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


Sequential Model 구성  
Spec:
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [3]:
model = keras.Sequential([
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [4]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 17s 7ms/step - loss: 0.2108 - accuracy: 0.9317 0s - loss: 0.2122 - accuracy
Epoch 2/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0332 - accuracy: 0.9901
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0189 - accuracy: 0.9941
Epoch 4/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0112 - accuracy: 0.9965
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0073 - accuracy: 0.9977
313/313 - 2s - loss: 0.0668 - accuracy: 0.9843


[0.06675118952989578, 0.9843000173568726]

### MNIST 데이터셋 : Functional API 활용

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [6]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


`keras.Input`으로 정의된 input및 output 레이어 구성을 통해 model을 구현  


Spec:
0. (28X28X1) 차원으로 정의된 Input
1. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. 64개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
3. Flatten 레이어
4. 128개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
5. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)


In [7]:
inputs = keras.Input(shape=(28, 28, 1))

x = keras.layers.Conv2D(32, 3, activation='relu')(inputs)
x = keras.layers.Conv2D(64, 3, activation='relu')(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(128, activation='relu')(x)
predictions = keras.layers.Dense(10, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [8]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2321 - accuracy: 0.9283
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0338 - accuracy: 0.9897
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0172 - accuracy: 0.9946
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0111 - accuracy: 0.9963
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0072 - accuracy: 0.9977
313/313 - 1s - loss: 0.0513 - accuracy: 0.9875


[0.0512717179954052, 0.987500011920929]

### MNIST 데이터셋 : Subclassing 활용


In [9]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [10]:
# 데이터 구성부분
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train=x_train[...,np.newaxis]
x_test=x_test[...,np.newaxis]

print(len(x_train), len(x_test))

60000 10000


`keras.Model` 을 상속받은 클래스를 만들자.  
`__init__()` 메소드 안에서 레이어를 선언하고, `call()` 메소드 안에서 forward propagation을 구현한다.  

_Functional 방식과 비교하자면, `call()`의 입력이 Input이고, `call()`의 리턴값이 Output이 되는 것_

In [11]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(32, 3, activation='relu')
        self.conv2 = keras.layers.Conv2D(64, 3, activation='relu')
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(128, activation='relu')
        self.fc2 = keras.layers.Dense(10, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()

In [12]:
# 모델 학습 설정

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.2049 - accuracy: 0.9367
Epoch 2/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0332 - accuracy: 0.9894
Epoch 3/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0164 - accuracy: 0.9945
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0124 - accuracy: 0.9955
Epoch 5/5
1875/1875 [==============================] - 14s 7ms/step - loss: 0.0090 - accuracy: 0.9967
313/313 - 1s - loss: 0.0566 - accuracy: 0.9860


[0.05664738640189171, 0.9860000014305115]

----
## Tensorflow2 API로 모델 구성: CIFAR-100 데이터셋

[CIFAR-100 dataset](https://www.cs.toronto.edu/~kriz/cifar.html)

###  CIFAR-100 데이터셋 : Sequential API 활용

In [13]:
import tensorflow as tf
from tensorflow import keras

In [14]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


Sequential Model 구성  


Spec:
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

In [15]:
model = keras.Sequential([
    keras.layers.Conv2D(16, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(100, activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 9s 5ms/step - loss: 3.9975 - accuracy: 0.0962
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9513 - accuracy: 0.2710
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6272 - accuracy: 0.3402
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3901 - accuracy: 0.3878
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2123 - accuracy: 0.4270
313/313 - 1s - loss: 2.6050 - accuracy: 0.3524


[2.604952812194824, 0.352400004863739]

### CIFAR-100 데이터셋 : Functional API 활용

In [17]:
import tensorflow as tf
from tensorflow import keras

In [18]:
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


Functional API를 활용한 Model 구성  


Spec:
0. (32X32X3) 차원으로 정의된 Input
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)

In [19]:
inputs = keras.Input(shape=(32, 32, 3))

x = keras.layers.Conv2D(16, 3, activation='relu')(inputs)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Conv2D(32, 3, activation='relu')(x)
x = keras.layers.MaxPool2D((2,2))(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(256, activation='relu')(x)
predictions = keras.layers.Dense(100, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=predictions)

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 3.9594 - accuracy: 0.1042
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9193 - accuracy: 0.2802
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6076 - accuracy: 0.3436
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.3695 - accuracy: 0.3943
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.1796 - accuracy: 0.4365
313/313 - 1s - loss: 2.5447 - accuracy: 0.3644


[2.5447332859039307, 0.3643999993801117]

### CIFAR-100 데이터셋 : Subclassing 활용

In [21]:
import tensorflow as tf
from tensorflow import keras

In [22]:
# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

50000 10000


Subclassing을 활용한 Model 구성  


Spec:
0. keras.Model 을 상속받았으며, __init__()와 call() 메소드를 가진 모델 클래스
1. 16개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
2. pool_size가 2인 MaxPool 레이어
3. 32개의 채널을 가지고, 커널의 크기가 3, activation function이 relu인 Conv2D 레이어
4. pool_size가 2인 MaxPool 레이어
5. 256개의 아웃풋 노드를 가지고, activation function이 relu인 Fully-Connected Layer(Dense)
6. 데이터셋의 클래스 개수에 맞는 아웃풋 노드를 가지고, activation function이 softmax인 Fully-Connected Layer(Dense)
7. call의 입력값이 모델의 Input, call의 리턴값이 모델의 Output


In [23]:
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')
        
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        
        return x
        
model = CustomModel()

In [24]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Epoch 1/5
1563/1563 [==============================] - 5s 3ms/step - loss: 4.0155 - accuracy: 0.0912
Epoch 2/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.9975 - accuracy: 0.2645
Epoch 3/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.6599 - accuracy: 0.3321
Epoch 4/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.4366 - accuracy: 0.3775
Epoch 5/5
1563/1563 [==============================] - 5s 3ms/step - loss: 2.2669 - accuracy: 0.4146
313/313 - 1s - loss: 2.6033 - accuracy: 0.3563


[2.6032817363739014, 0.3562999963760376]

----
## GradientTape 활용

아래에서는 이전 스텝에서 진행했던 학습을 `tf.GradientTape` 를 이용한 것으로 변형했다.

In [25]:
import tensorflow as tf
from tensorflow import keras

# 데이터 구성부분
cifar100 = keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
print(len(x_train), len(x_test))

# 모델 구성부분
class CustomModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.conv1 = keras.layers.Conv2D(16, 3, activation='relu')
        self.maxpool1 = keras.layers.MaxPool2D((2,2))
        self.conv2 = keras.layers.Conv2D(32, 3, activation='relu')
        self.maxpool2 = keras.layers.MaxPool2D((2,2))
        self.flatten = keras.layers.Flatten()
        self.fc1 = keras.layers.Dense(256, activation='relu')
        self.fc2 = keras.layers.Dense(100, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)

        return x

model = CustomModel()

50000 10000


- `tape.gradient()`를 통해 매 스텝 학습이 진행될 때마다 발생하는 그래디언트를 추출한 후 
- `optimizer.apply_gradients()`를 통해 발생한 그래디언트가 업데이트해야 할 파라미터 `model.trainable_variables` 지정

In [26]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

# tf.GradientTape()를 활용한 train_step
def train_step(features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

In [27]:
import time
def train_model(batch_size=32):   # model.fit() 메소드와 기능적으로 동일
    start = time.time()
    for epoch in range(5):
        x_batch = []
        y_batch = []
        for step, (x, y) in enumerate(zip(x_train, y_train)):
            if step % batch_size == batch_size-1:
                x_batch.append(x)
                y_batch.append(y)
                loss = train_step(np.array(x_batch, dtype=np.float32), np.array(y_batch, dtype=np.float32))
                x_batch = []
                y_batch = []
        print('Epoch %d: last batch loss = %.4f' % (epoch, float(loss)))
    print("It took {} seconds".format(time.time() - start))

train_model()

Epoch 0: last batch loss = 4.5313
Epoch 1: last batch loss = 4.4994
Epoch 2: last batch loss = 4.5539
Epoch 3: last batch loss = 4.5735
Epoch 4: last batch loss = 4.3571
It took 93.1763801574707 seconds


`tf.GradientTape()`를 활용하면 `model.compile()`과 `model.fit()` 안에 감추어져 있던 한 스텝의 학습 단계(위 예제에서는 train_step 메소드)를 끄집어내서 자유롭게 재구성할 수 있게 된다.  

그동안 흔히 다루어 왔던 지도학습 방식과 다른 강화학습 또는 GAN(Generative Advasarial Network)의 학습을 위해서는 train_step 메소드의 재구성이 필수적이다.

In [28]:
# evaluation
prediction = model.predict(x_test, batch_size=x_test.shape[0], verbose=1)
temp = sum(np.squeeze(y_test) == np.argmax(prediction, axis=1))
temp/len(y_test)  # Accuracy

1/1 [==============================] - 3s 3s/step


0.0633